In [1]:
import random as rnd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import chisquare, gmean, gstd

In [2]:
class my_random_class():
  def __init__(self, a=2**18+1, c=7, m = 2**32):
    #-----full spectra properties---------#
    self.c = c
    self.a = a
    self.m = m

  def seed(self, val):
    self.r = val

  def random(self):
    r = (self.a * self.r +self.c) % self.m
    self.r = r
    return r

In [3]:
def chi_squared(test_bins):
    """ The Chi-Squared (equidistant) function from the problem statement. """
    n = len(test_bins)
    N = 0
    chi_squared_reduced = 0

    for i in range(len(test_bins)):
        N += test_bins[i]

    for i in range(len(test_bins)):
        chi_squared_reduced += (test_bins[i]-(N/n))**2

    chi_squared = (n/N)*chi_squared_reduced
    return chi_squared

In [9]:
def test_loop(label, rnd_func, chi_sq_samples=100, n=10, N=10000, m=2**32, needs_normalization=True):
    """ 
    Generate 100 chi-squared values and return the mean and standard deviation.
    Chi-squared values are computed...
    """
    # print("- " * 40)
    # print(f"Testing {label} RNG...")

    chisq_vals = []
    chisq_scipy_vals = []
    for chi_ndx in range(chi_sq_samples):
        random_vals = [rnd_func.random() for i in range(N)]
        if needs_normalization: random_vals = np.array(random_vals) / m    # normalize the vals1 array [0,1)
        # print(f"{label} random values: {random_vals[:5]}")

        # bin the normalized random values
        hist, hist_bins = np.histogram(random_vals, bins=n, range=(0,1))
        # print(f"{label} histogram: {hist}")

        # compute the chi-squared statistic for each RNG
        chi_sq = chi_squared(hist); chi_sq_scipy = chisquare(hist) # compare to scipy
        # print(f"{label} chi_sq: {chi_sq} (scipy: {chi_sq_scipy})")
        chisq_vals.append(chi_sq)
        chisq_scipy_vals.append(chi_sq_scipy)
    
    # compute the mean and standard deviation of the chi-squared values
    chisq_mean = gmean(chisq_vals)
    chisq_std = gstd(chisq_vals)
    chisq_scipy_mean = gmean(chisq_scipy_vals)[0]
    chisq_scipy_std = gstd(chisq_scipy_vals)[0]
    print(f"{label} chi_sq mean: {chisq_mean}, std: {chisq_std} (scipy: {chisq_scipy_mean}, {chisq_scipy_std})")

    return chisq_mean, chisq_std


In [11]:
# implement 3 different RNGs (f_a, f_b, f_c)
rnd_a = my_random_class(a=2**18+1, c=7)
rnd_a.seed(314159)
rnd_b = my_random_class(a=1812433253, c=7)
rnd_b.seed(314159)
rnd_c = rnd # python's built-in RNG
rnd_c.seed(314159)

m = 2**32
N = 10000
n = 10

print("- " * 40)
print(f"Results with n = {n}")

mean, stddev = test_loop("rnd_a", rnd_a, chi_sq_samples=100, n=n, N=N, m=m)
print(f"rnd_a mean: {mean} (std: {stddev})")

mean, stddev = test_loop("rnd_b", rnd_b, chi_sq_samples=100, n=n, N=N, m=m)
print(f"rnd_b mean: {mean} (std: {stddev})")

mean, stddev = test_loop("rnd_c", rnd_c, chi_sq_samples=100, n=n, N=N, m=m, needs_normalization=False)
print(f"rnd_c mean: {mean} (std: {stddev})")

n = 100
print("- " * 40)
print(f"Results with n = {n}")

mean, stddev = test_loop("rnd_a", rnd_a, chi_sq_samples=100, n=n, N=N, m=m)
print(f"rnd_a mean: {mean},  std: {stddev}")

mean, stddev = test_loop("rnd_b", rnd_b, chi_sq_samples=100, n=n, N=N, m=m)
print(f"rnd_b mean: {mean},  std: {stddev}")

mean, stddev = test_loop("rnd_c", rnd_c, chi_sq_samples=100, n=n, N=N, m=m, needs_normalization=False)
print(f"rnd_c mean: {mean},  std: {stddev}")



- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Results with n = 10
rnd_a chi_sq mean: 6.304125714175964, std: 1.9039521800926407 (scipy: 6.304125714175963, 1.9039521800926404)
rnd_a mean: 6.304125714175964 (std: 1.9039521800926407)
rnd_b chi_sq mean: 7.912732724025049, std: 1.6274476761094778 (scipy: 7.912732724025049, 1.6274476761094778)
rnd_b mean: 7.912732724025049 (std: 1.6274476761094778)
rnd_c chi_sq mean: 8.04892399053811, std: 1.5294356715664328 (scipy: 8.048923990538114, 1.5294356715664328)
rnd_c mean: 8.04892399053811 (std: 1.5294356715664328)
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Results with n = 100
rnd_a chi_sq mean: 63.287572897357705, std: 1.5997153565338305 (scipy: 63.28757289735765, 1.599715356533831)
rnd_a mean: 63.287572897357705,  std: 1.5997153565338305
rnd_b chi_sq mean: 98.7657467693906, std: 1.1664593970135526 (scipy: 98.7657467693905, 1.1664593970135524)
rnd_b mean: 98.7657467693906,  